In [1]:
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
import json, re, pymongo
from config import ConfigData

In [2]:
llm_openai = ChatOpenAI(openai_api_key=ConfigData.OPEN_AI_KEY, model='gpt-3.5-turbo-0125')

In [3]:
table_schema = ConfigData.TABLE_SCHEMA
schema_description = ConfigData.SCHEMA_DESCRIPTION
json_ex_1 = ConfigData.FEW_SHOT_EXAMPLE_1

In [4]:
json_ex_string = json.dumps(json_ex_1)

In [5]:
prompt_template_for_creating_query = """
    You are an expert in crafting NoSQL queries for MongoDB with 10 years of experience, particularly in MongoDB. 
    I will provide you with the table_schema and schema_description in a specified format. 
    Your task is to read the user_question, which will adhere to certain guidelines or formats, and create a NOSQL MongoDb pipeline accordingly.

    Table schema:""" +  table_schema + """
    Schema Description: """ + schema_description + """

    Here are some example:
    Input: name of departments where number of employees is greater than 1000
    Output: {json_ex_string_1} 

    Note: You have to just return the query nothing else. Don't return any additional text with the query.
    Input: {user_question}
    """
    
query_creation_prompt = PromptTemplate(
    template=prompt_template_for_creating_query,
    input_variables=["user_question", "json_ex_string_1"],
)

In [6]:
llmchain = LLMChain(llm=llm_openai, prompt=query_creation_prompt, verbose=True)

In [7]:
def get_query(user_question):
    response = llmchain.invoke({
        "user_question": user_question,
        "json_ex_string_1": json_ex_string
    })

    response_text = response['text'].replace("Output: ", "")
    pattern = r'db\.collectionName\.aggregate\(\s*\['

    output_string = re.sub(pattern, '', response_text)

    return json.loads(output_string)

In [8]:
client = pymongo.MongoClient(ConfigData.MONGO_DB_URI)
db = client[ConfigData.DB_NAME]
collection_name = db[ConfigData.COLLECTION_NAME]

In [9]:
query_1 = get_query(user_question="when InnovateTech Solutions founded?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are an expert in crafting NoSQL queries for MongoDB with 10 years of experience, particularly in MongoDB. 
    I will provide you with the table_schema and schema_description in a specified format. 
    Your task is to read the user_question, which will adhere to certain guidelines or formats, and create a NOSQL MongoDb pipeline accordingly.

    Table schema:
                    "_id": "string",
                    "name": "string",
                    "industry": "string",
                    "founded_year": "string",
                    "employees": "string",
                    "hq_location": "string",
                    "products": "array of strings",
                    "revenue": "string",
                    "ceo": 
                        "name": "string",
                        "age": "string",
                        "email": "string"
                    ,
                    "departments": [
            

In [10]:
pipeline = query_1
result = collection_name.aggregate(pipeline)
for doc in result:
    print(doc)

{'_id': ObjectId('66165f917b226d174b553af4'), 'founded_year': 2008}
